# News Mood Challenge

In [1]:
#import necessary modules and info
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

In [2]:
#authenticate tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
#compile list of twitter handles through which to loop
handles = ["@BBCNews", "@CBSNews", "@CNN", "@FoxNews", "@nytimes"]

#create a list to hold dictionaries of the sentiment analysis resules
s_scores = []

#create a variable to store the oldest tweet id for the loop
ot = None

In [6]:
#loop through the news organizations
for handle in handles:
    
    #create counter for tabulating tweets
    c = 1
    
    #loop through 5 pages (20 tweets per page) of tweets to acquire 100 tweets
    for x in range(5):
        tweets = api.user_timeline(handle, max_id = ot)
    
        #loop through the tweets themselves
        for t in tweets:
            
            #run Vader analysis on the tweet
            scores = analyzer.polarity_scores(t["text"])
            
            #add dictionary of scores to the list along with the media source date stamp
            s_scores.append({"Media Source": t["user"]["name"],
                            "Date": t["created_at"], 
                           "Compound": scores["compound"],
                           "Positive": scores["pos"],
                           "Negative": scores["neu"],
                           "Neutral": scores["neg"],
                           "Tweets Ago": c})
            
            # store the tweet id in the oldest tweet variable and subtract 1 to continue iteration
            ot = t['id'] - 1
            
            #increase the counter to document the next tweet
            c += 1            

In [15]:
#convert the list of dictionaries containing the scores into a DataFrame
spd = pd.DataFrame(s_scores)
spd = spd[["Media Source", "Date", "Tweets Ago", "Compound", "Negative", "Neutral", "Positive"]]
spd.head()

,Media Source,Date,Tweets Ago,Compound,Negative,Neutral,Positive
0,BBC News (UK),Wed Jul 25 22:15:45 +0000 2018,1,0.0,1.0,0.0,0.0
1,BBC News (UK),Wed Jul 25 21:58:52 +0000 2018,2,0.0,1.0,0.0,0.0
2,BBC News (UK),Wed Jul 25 21:55:10 +0000 2018,3,0.0,1.0,0.0,0.0
3,BBC News (UK),Wed Jul 25 21:55:08 +0000 2018,4,0.0,1.0,0.0,0.0
4,BBC News (UK),Wed Jul 25 21:55:06 +0000 2018,5,0.0,1.0,0.0,0.0


In [11]:
#export the DataFrame to a csv
spd.to_csv("100tweets_by_org.csv")